In [1]:
pip install pinecone-client

   ---------------------------------------- 0.0/216.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/216.4 kB ? eta -:--:--
   ----------------- ---------------------- 92.2/216.4 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 216.4/216.4 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\Chringo\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="7c6028ed-8d2a-4003-8e7f-fca445aad9a1")

In [3]:
# List already created Indexes
pc.list_indexes()

{'indexes': []}

In [4]:
# Create new Index
index_name = "docs-quickstart-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=2,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    )
pc.list_indexes()

{'indexes': [{'dimension': 2,
              'host': 'docs-quickstart-index-3a68ps2.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'docs-quickstart-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [5]:
# Add Vectors to the created Index and put them in 2 different namespaces
index = pc.Index(index_name)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, 1.5]},
        {"id": "vec2", "values": [2.0, 1.0]},
        {"id": "vec3", "values": [0.1, 3.0]},
    ],
    namespace="ns1"
)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, -2.5]},
        {"id": "vec2", "values": [3.0, -2.0]},
        {"id": "vec3", "values": [0.5, -1.5]},
    ],
    namespace="ns2"
)

{'upserted_count': 3}

In [7]:
print(index.describe_index_stats())

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 3}, 'ns2': {'vector_count': 3}},
 'total_vector_count': 6}


In [9]:
# Peform a similarity search for this query
query_results1 = index.query(
    namespace="ns1",
    vector=[1.0, 1.5],
    top_k=3,
    include_values=True
)

print(query_results1)

query_results2 = index.query(
    namespace="ns2",
    vector=[1.0,-2.5],
    top_k=3,
    include_values=False
)

print(query_results2)

{'matches': [{'id': 'vec1', 'score': 1.0, 'values': []},
             {'id': 'vec2', 'score': 0.868243158, 'values': []},
             {'id': 'vec3', 'score': 0.850068152, 'values': []}],
 'namespace': 'ns1',
 'usage': {'read_units': 5}}
{'matches': [{'id': 'vec1', 'score': 1.0, 'values': [1.0, -2.5]},
             {'id': 'vec3', 'score': 0.998274386, 'values': [0.5, -1.5]},
             {'id': 'vec2', 'score': 0.824041963, 'values': [3.0, -2.0]}],
 'namespace': 'ns2',
 'usage': {'read_units': 6}}


In [ ]:
# If yout want to clean up and delete the index
pc.delete_index(index_name)